Upload dataset and unzip the file

In [489]:
!tar xzf lingspam_public.tar.gz

Import library

In [490]:
from decimal import *
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from collections import Counter
from collections import OrderedDict
from sklearn import svm

Load words in emails into a word list.

In [491]:
train_path = './lingspam_public/lemm_stop'  # parent directory of dataset we use
paths = [os.path.join(train_path, folders) for folders in os.listdir(train_path)]  
all_words = [] 
labels = []  # 1 for spam, 0 for legit
test_path = ""
for path in paths:
    if(path[-2:] == '10'):  # path[-2:] check if this is part10 which is used for testing, skip 
        test_path = path
        continue
    files = [os.path.join(path, f) for f in os.listdir(path)]  # get file names of this part and generate path to files
    for email in files:
        if(email.find('spmsg') != -1): #check filenames to check if it is spam 
            labels.append(1)
        else:
            labels.append(0)
        
        with open(email) as m:
            for i, content in enumerate(m):  # i: line number , content: content of each line
                if(i == 2):  # open txt and find content is in the 3rd line
                    words = content.split()
                    all_words += words

word_dict = Counter(all_words)  

# remove symbols which have no big relationship with spam or legit
for word in list(word_dict.keys()):
    if(word.isalpha() == False):
        del word_dict[word]

# distinct words
dist_words = list(word_dict.keys())

# Compute Information Gain(IG)

1. compute H(C), C is RV that determins if a document is spam or legit 

In [492]:
spam = 0
legit = 0
for label in labels:
  if label == 1:
    spam += 1
  else:
    legit += 1  

print(f"spam email: {spam}, legit email: {legit}, total email {spam + legit}")
p_spam = spam / (spam + legit)

H_c = -1 * ( p_spam*np.log2(p_spam) + (1-p_spam)*np.log2(1-p_spam) )
print('H(C): ', H_c)

spam email: 432, legit email: 2170, total email 2602
H(C):  0.6485330171848536


2. Compute H(C|Xi), need to create dictionary for word:#spam and word:#legit to help computing H(C|Xi) 

In [493]:
word_spam = {}
word_legit = {}
for word in dist_words:
    word_spam[word] = 0
    word_legit[word] = 0

for path in paths:
    if(path[-2:] == '10'):  
        continue
    files = [os.path.join(path, f) for f in os.listdir(path)]  
    for email in files:
        if(email.find('spmsg') != -1):  
            is_spam = True
        else:
            is_spam = False
        dist_set = set() # there could be repeated word, we only need  occurance, use set
        with open(email) as m:
            for i, content in enumerate(m):  
                if(i == 2):  
                    words = content.split()
                    for word in words:
                        if(word.isalpha() == False): # skip symbols
                            continue
                        else:
                            dist_set.add(word)
        for e in dist_set:
            if(is_spam == True):
                word_spam[e] += 1
            else:
                word_legit[e] += 1

Compute H(C|Xi) = "combination" of P(X=x,C=c)log(P(C=c|X=x))

In [494]:
H_cx = {}
for word in dist_words:
    H_cx[word] = 0 
    # c = legit, x = 0 
    if ((legit - word_legit[word]) != 0 and (spam - word_spam[word] + legit - word_legit[word]) != 0): #sum only when non-zero probability
        H_cx[word] += -1 * (legit - word_legit[word]) / (spam + legit) * np.log2( (legit - word_legit[word]) / (spam - word_spam[word] + legit - word_legit[word]) )
    # c = legit, x = 1 
    if ((word_legit[word]) != 0 and (word_spam[word] + word_legit[word]) != 0):
        H_cx[word] += -1 * (word_legit[word]) / (spam + legit) * np.log2( word_legit[word] / (word_spam[word] + word_legit[word]) )
    # c = spam, x = 0 
    if (spam-word_spam[word] != 0 and (spam - word_spam[word] + legit - word_legit[word]) != 0):
        H_cx[word] += -1 * (spam - word_spam[word]) / (spam + legit) * np.log2( (spam - word_spam[word] ) / (spam - word_spam[word] + legit - word_legit[word]) ) 
    # c = spam, x = 1 
    if (word_spam[word] != 0 and word_spam[word] + word_legit[word] != 0):
        H_cx[word] += -1 * (word_spam[word]) / (spam + legit) * np.log2( word_spam[word] / (word_spam[word] + word_legit[word]) ) 
        

3. Compute IG

In [495]:
IG = {}
for word in list(H_cx.keys()):
    IG[word] = H_c - H_cx[word]

# Get top N features

In [496]:
# sort by value
# sorted can be applied to iterable containers
# sorted return a new result rather than sort in place
# sorted return a list object not a sorted dictionary!!!!!!!
# sorted_IG = sorted(IG.items(), key=lambda x : x[1], reverse=True)
sorted_IG = OrderedDict(sorted(IG.items(), key=lambda x : x[1], reverse=True))
f10 = list(sorted_IG.keys())[:10]  # top 10
f100 = list(sorted_IG.keys())[:100]  # top 100
f1000 = list(sorted_IG.keys())[:1000]  # top 1000

# Part1 report: top-10 words:

In [497]:
print("Top-10 words:")
c=0
for i in sorted_IG.items():
  c+=1
  if c > 10:
    break
  print(i[0],":",i[1])

Top-10 words:
language : 0.19967490044495867
remove : 0.16891307295882557
free : 0.15997268664519282
linguistic : 0.1462359533473594
university : 0.1450067285566311
money : 0.11867735600192031
click : 0.10117445536770531
market : 0.09149985619225265
our : 0.08706843935925757
business : 0.08301653103351447


# Bernoulli NB classifier with binary feature

*Load test data*

In [498]:
test_label = []  
f10_test_bbf = []  # test emails' feature matrix for top 10
f100_test_bbf = []  # test emails' feature matrix for top 100
f1000_test_bbf = []  # test emails' feature matrix for top 1000
emails = [os.path.join(test_path, file_name) for file_name in os.listdir(test_path)]  
for email in emails:
    # spam: 1, legit: 0
    if email.find('spmsg')!=-1:
        test_label.append(1)
    else:
        test_label.append(0)
    
    with open(email) as m:
        temp_f10_bf = []
        temp_f100_bf = []
        temp_f1000_bf = []

        for i, content in enumerate(m):  
            if i==2:  # content at 3rd line
                words_in_email = set(content.split())
                for w in f10:
                    if w in words_in_email:
                        temp_f10_bf.append(1)
                    else:
                        temp_f10_bf.append(0)
                for w in f100:
                    if w in words_in_email:
                        temp_f100_bf.append(1)
                    else:
                        temp_f100_bf.append(0)
                for w in f1000:
                    if w in words_in_email:
                        temp_f1000_bf.append(1)
                    else:
                        temp_f1000_bf.append(0)

        f10_test_bbf.append(temp_f10_bf)
        f100_test_bbf.append(temp_f100_bf)
        f1000_test_bbf.append(temp_f1000_bf)


**1. top 10 feature**

In [499]:
N = 10

1.1 Compute probability

In [500]:
# x is conbination of words
# compute P(x|spam)
p_x_spam_10 = []
for case in f10_test_bbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_spam[f10[i]] + 1) / (spam + 2) # with laplacian smoothing
        else:
            p *= (1 - (word_spam[f10[i]] + 1) / (spam + 2)) # with laplacian smoothing
    p_x_spam_10.append(p)    

# compute P(x|legit)
p_x_legit_10 = []
for case in f10_test_bbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_legit[f10[i]] + 1) / (legit + 2) # with laplacian smoothing
        else:
            p *= 1 - (word_legit[f10[i]] + 1) / (legit + 2) # with laplacian smoothing
    p_x_legit_10.append(p)    

# compute P(spam)
p_spam = spam/(spam+legit)

# compute P(legit)
p_legit = legit/(spam+legit) 

# compute P(x)
p_x_10 = []
for i in range(len(f10_test_bbf)):
    p = p_spam * p_x_spam_10[i] + p_legit * p_x_legit_10[i]
    p_x_10.append(p)
 
  

1.2 Classify by computing P(spam|X) and P(legit|X) and compare

In [501]:
f10_pre = []
p_spam_x_10 = []
p_legit_x_10 = []
for i in range(len(f10_test_bbf)):
    psx = p_x_spam_10[i] * p_spam / p_x_10[i]
    p_spam_x_10.append(psx)
    plx = p_x_legit_10[i] * p_legit / p_x_10[i]
    p_legit_x_10.append(plx)
    if psx >= plx :
        f10_pre.append(1)
    else:
        f10_pre.append(0)

1.3 Report result

In [502]:
test_label[:10]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [503]:
f10_pre[:10]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [504]:
#p_spam_x_10[:10]

In [505]:
#p_legit_x_10[:10]

In [506]:
correct = 0;
total = len(test_label)
for i in range(total):
    if test_label[i] == f10_pre[i]:
        correct+=1
correct_rate = correct / total
print("correct rate: ",correct_rate)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if f10_pre[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_bbf_10 = true_spam_in_spam / total_pre_spam   
print("spam precision: ",spam_precision_bbf_10)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if f10_pre[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_bbf_10 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_bbf_10)

correct rate:  0.9518900343642611
spam precision:  0.8888888888888888
spam_recall:  0.8163265306122449


**2. top 100 feature**

In [507]:
N = 100

2.1 Compute probability

In [508]:
# x is conbination of words
# compute P(x|spam)
p_x_spam_100 = []
for case in f100_test_bbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_spam[f100[i]] + 1) / (spam + 2) # with laplacian smoothing
        else:
            p *= (1 - (word_spam[f100[i]] + 1) / (spam + 2)) # with laplacian smoothing
    p_x_spam_100.append(p)    

# compute P(x|legit)
p_x_legit_100 = []
for case in f100_test_bbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_legit[f100[i]] + 1) / (legit + 2) # with laplacian smoothing
        else:
            p *= 1 - (word_legit[f100[i]] + 1) / (legit + 2) # with laplacian smoothing
    p_x_legit_100.append(p)    

# compute P(spam)
p_spam = spam/(spam+legit)

# compute P(legit)
p_legit = legit/(spam+legit) 

# compute P(x)
p_x_100 = []
for i in range(len(f100_test_bbf)):
    p = p_spam * p_x_spam_100[i] + p_legit * p_x_legit_100[i]
    p_x_100.append(p)
 
#print(len(p_x_spam_100))
#print(len(p_x_100))

2.2 Classify by computing P(spam|X) and P(legit|X) and compare

In [509]:
f100_pre = []
p_spam_x_100 = []
p_legit_x_100 = []
for i in range(len(f100_test_bbf)):
    psx = p_x_spam_100[i] * p_spam / p_x_100[i]
    p_spam_x_100.append(psx)
    plx = p_x_legit_100[i] * p_legit / p_x_100[i]
    p_legit_x_100.append(plx)
    if psx >= plx :
        f100_pre.append(1)
    else:
        f100_pre.append(0)

2.3 Report result

In [510]:
test_label[:10]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [511]:
f100_pre[:10]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [512]:
correct = 0;
total = len(test_label)
for i in range(total):
    if test_label[i] == f100_pre[i]:
        correct+=1
correct_rate = correct / total
print("correct rate: ",correct_rate)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if f100_pre[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_bbf_100 = true_spam_in_spam / total_pre_spam   
print("spam precision: ",spam_precision_bbf_100)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if f100_pre[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_bbf_100 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_bbf_100)

correct rate:  0.9450171821305842
spam precision:  1.0
spam_recall:  0.673469387755102


3. top 100 feature


In [513]:
N = 1000

**3.1 Compute probability**

In [514]:
# x is conbination of words
# compute P(x|spam)
p_x_spam_1000 = []
for case in f1000_test_bbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_spam[f1000[i]] + 1) / (spam + 2) # with laplacian smoothing
        else:
            p *= (1 - (word_spam[f1000[i]] + 1) / (spam + 2)) # with laplacian smoothing
    p_x_spam_1000.append(p)    

# compute P(x|legit)
p_x_legit_1000 = []
for case in f1000_test_bbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_legit[f1000[i]] + 1) / (legit + 2) # with laplacian smoothing
        else:
            p *= 1 - (word_legit[f1000[i]] + 1) / (legit + 2) # with laplacian smoothing
    p_x_legit_1000.append(p)    

# compute P(spam)
p_spam = spam/(spam+legit)

# compute P(legit)
p_legit = legit/(spam+legit) 

# compute P(x)
p_x_1000 = []
for i in range(len(f1000_test_bbf)):
    p = p_spam * p_x_spam_1000[i] + p_legit * p_x_legit_1000[i]
    p_x_1000.append(p)
 
#print(len(p_x_spam_1000))
#print(len(p_x_1000))

3.2 Classify by computing P(spam|X) and P(legit|X) and compare

In [515]:
f1000_pre = []
p_spam_x_1000 = []
p_legit_x_1000 = []
for i in range(len(f1000_test_bbf)):
    psx = p_x_spam_1000[i] * p_spam / p_x_1000[i]
    p_spam_x_1000.append(psx)
    plx = p_x_legit_1000[i] * p_legit / p_x_1000[i]
    p_legit_x_1000.append(plx)
    if psx >= plx :
        f1000_pre.append(1)
    else:
        f1000_pre.append(0)

3.3 Report result

In [516]:
test_label[:10]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [517]:
f1000_pre[:10]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [518]:
correct = 0;
total = len(test_label)
for i in range(total):
    if test_label[i] == f1000_pre[i]:
        correct+=1
correct_rate = correct / total
print("correct rate: ",correct_rate)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if f1000_pre[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_bbf_1000 = true_spam_in_spam / total_pre_spam   
print("spam precision: ",spam_precision_bbf_1000)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if f1000_pre[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_bbf_1000 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_bbf_1000)

correct rate:  0.9347079037800687
spam precision:  1.0
spam_recall:  0.6122448979591837


***It seems that spam precision goes up while spam recall goes down with more feature selected***

# Multinomial NB with binary features

*Load test data*

In [519]:
test_label = []  
f10_test_mbf = []  # test emails' feature matrix for top 10
f100_test_mbf = []  # test emails' feature matrix for top 100
f1000_test_mbf = []  # test emails' feature matrix for top 1000
emails = [os.path.join(test_path, file_name) for file_name in os.listdir(test_path)]  
for email in emails:
    # spam: 1, legit: 0
    if email.find('spmsg')!=-1:
        test_label.append(1)
    else:
        test_label.append(0)
    
    with open(email) as m:
        temp_f10_bf = []
        temp_f100_bf = []
        temp_f1000_bf = []

        for i, content in enumerate(m):  
            if i==2:  # content at 3rd line
                words_in_email = set(content.split())
                for w in f10:
                    if w in words_in_email:
                        temp_f10_bf.append(1)
                    else:
                        temp_f10_bf.append(0)
                for w in f100:
                    if w in words_in_email:
                        temp_f100_bf.append(1)
                    else:
                        temp_f100_bf.append(0)
                for w in f1000:
                    if w in words_in_email:
                        temp_f1000_bf.append(1)
                    else:
                        temp_f1000_bf.append(0)

        f10_test_mbf.append(temp_f10_bf)
        f100_test_mbf.append(temp_f100_bf)
        f1000_test_mbf.append(temp_f1000_bf)


**1. top 10 feature**

In [520]:
N = 10

1.1 Compute probability

In [521]:
# compute P(x|spam)
p_x_spam_10 = []
for case in f10_test_mbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_spam[f10[i]] + 1) / (spam + 2) # with laplacian smoothing
        else:
            p *= 1 # different from bernoulli: ignore negative evidence
    p_x_spam_10.append(p)    

# compute P(x|legit)
p_x_legit_10 = []
for case in f10_test_mbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_legit[f10[i]] + 1) / (legit + 2) # with laplacian smoothing
        else:
            p *= 1 # different from bernoulli: ignore negative evidence
    p_x_legit_10.append(p)    

# compute P(spam)
p_spam = spam/(spam+legit)

# compute P(legit)
p_legit = legit/(spam+legit) 

# compute P(x)
p_x_10 = []
for i in range(len(f10_test_mbf)):
    p = p_spam * p_x_spam_10[i] + p_legit * p_x_legit_10[i]
    p_x_10.append(p)
 
#print(len(p_x_spam_10))
#print(len(p_x_10))

1.2 Classify by computing P(spam|X) and P(legit|X) and compare

In [522]:
f10_pre = []
p_spam_x_10 = []
p_legit_x_10 = []
for i in range(len(f10_test_mbf)):
    psx = p_x_spam_10[i] * p_spam / p_x_10[i]
    p_spam_x_10.append(psx)
    plx = p_x_legit_10[i] * p_legit / p_x_10[i]
    p_legit_x_10.append(plx)
    if psx >= plx :
        f10_pre.append(1)
    else:
        f10_pre.append(0)

1.3 report result

In [523]:
correct = 0;
total = len(test_label)
for i in range(total):
    if test_label[i] == f10_pre[i]:
        correct+=1
correct_rate = correct / total
print("correct rate: ",correct_rate)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if f10_pre[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_mbf_10 = true_spam_in_spam / total_pre_spam   
print("spam precision: ",spam_precision_mbf_10)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if f10_pre[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_mbf_10 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_mbf_10)

correct rate:  0.9484536082474226
spam precision:  0.8695652173913043
spam_recall:  0.8163265306122449


**2. top 100 feature**

In [524]:
N = 100

2.1 Compute probability

In [525]:
# compute P(x|spam)
p_x_spam_100 = []
for case in f100_test_mbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_spam[f100[i]] + 1) / (spam + 2) # with laplacian smoothing
        else:
            p *= 1 # different from bernoulli: ignore negative evidence 
    p_x_spam_100.append(p)    

# compute P(x|legit)
p_x_legit_100 = []
for case in f100_test_mbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_legit[f100[i]] + 1) / (legit + 2) # with laplacian smoothing
        else:
            p *= 1 # different from bernoulli: ignore negative evidence
    p_x_legit_100.append(p)    

# compute P(spam)
p_spam = spam/(spam+legit)

# compute P(legit)
p_legit = legit/(spam+legit) 

# compute P(x)
p_x_100 = []
for i in range(len(f100_test_mbf)):
    p = p_spam * p_x_spam_100[i] + p_legit * p_x_legit_100[i]
    p_x_100.append(p)
 
#print(len(p_x_spam_100))
#print(len(p_x_100))

2.2 Classify by computing P(spam|X) and P(legit|X) and compare

In [526]:
f100_pre = []
p_spam_x_100 = []
p_legit_x_100 = []
for i in range(len(f100_test_mbf)):
    psx = p_x_spam_100[i] * p_spam / p_x_100[i]
    p_spam_x_10.append(psx)
    plx = p_x_legit_100[i] * p_legit / p_x_100[i]
    p_legit_x_100.append(plx)
    if psx >= plx :
        f100_pre.append(1)
    else:
        f100_pre.append(0)

2.3 Report result

In [527]:
correct = 0;
total = len(test_label)
for i in range(total):
    if test_label[i] == f100_pre[i]:
        correct+=1
correct_rate = correct / total
print("correct rate: ",correct_rate)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if f100_pre[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_mbf_100 = true_spam_in_spam / total_pre_spam   
print("spam precision: ",spam_precision_mbf_100)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if f100_pre[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_mbf_100 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_mbf_100)

correct rate:  0.9759450171821306
spam precision:  0.9038461538461539
spam_recall:  0.9591836734693877


**3. top 1000 feature**

In [528]:
N = 1000

3.1 Compute probability

In [529]:
# compute P(x|spam)
p_x_spam_1000 = []
for case in f1000_test_mbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_spam[f1000[i]] + 1) / (spam + 2) # with laplacian smoothing
        else:
            p *= 1 # different from bernoulli: ignore negative evidence 
    p_x_spam_1000.append(p)    

# compute P(x|legit)
p_x_legit_1000 = []
for case in f1000_test_mbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_legit[f1000[i]] + 1) / (legit + 2) # with laplacian smoothing
        else:
            p *= 1 # different from bernoulli: ignore negative evidence
    p_x_legit_1000.append(p)    

# compute P(spam)
p_spam = spam/(spam+legit)

# compute P(legit)
p_legit = legit/(spam+legit) 

# compute P(x)
p_x_1000 = []
for i in range(len(f1000_test_mbf)):
    p = p_spam * p_x_spam_1000[i] + p_legit * p_x_legit_1000[i]
    p_x_1000.append(p)
 
#print(len(p_x_spam_1000))
#print(len(p_x_1000))

3.2 Classify by computing P(spam|X) and P(legit|X) and compare

In [530]:
f1000_pre = []
p_spam_x_1000 = []
p_legit_x_1000 = []
for i in range(len(f1000_test_mbf)):
    psx = p_x_spam_1000[i] * p_spam / p_x_1000[i]
    p_spam_x_1000.append(psx)
    plx = p_x_legit_1000[i] * p_legit / p_x_1000[i]
    p_legit_x_1000.append(plx)
    if psx >= plx :
        f1000_pre.append(1)
    else:
        f1000_pre.append(0)

3.3 Report result

In [531]:
correct = 0;
total = len(test_label)
for i in range(total):
    if test_label[i] == f1000_pre[i]:
        correct+=1
correct_rate = correct / total
print("correct rate: ",correct_rate)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if f1000_pre[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_mbf_1000 = true_spam_in_spam / total_pre_spam   
print("spam precision: ",spam_precision_mbf_1000)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if f1000_pre[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_mbf_1000 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_mbf_1000)

correct rate:  0.9896907216494846
spam precision:  1.0
spam_recall:  0.9387755102040817


***Adding feature seems helps multinomial NB with binary feature positively***

# Multinomial NB with term frequency (TF) features

*Load test data*

In [532]:
test_label = []  
f10_test_mtf = []  # test emails' feature matrix for top 10
f100_test_mtf = []  # test emails' feature matrix for top 100
f1000_test_mtf = []  # test emails' feature matrix for top 1000
emails = [os.path.join(test_path, file_name) for file_name in os.listdir(test_path)]  
for email in emails:
    # spam: 1, legit: 0
    if email.find('spmsg')!=-1:
        test_label.append(1)
    else:
        test_label.append(0)
    
    with open(email) as m:
        temp_f10_tf = []
        temp_f100_tf = []
        temp_f1000_tf = []

        for i, content in enumerate(m):  
            if i==2:  # content at 3rd line
                words_in_email = set(content.split())
                for w in f10:
                    if w in words_in_email:
                        temp_f10_tf.append(1)
                    else:
                        temp_f10_tf.append(0)
                for w in f100:
                    if w in words_in_email:
                        temp_f100_tf.append(1)
                    else:
                        temp_f100_tf.append(0)
                for w in f1000:
                    if w in words_in_email:
                        temp_f1000_tf.append(1)
                    else:
                        temp_f1000_tf.append(0)

        f10_test_mtf.append(temp_f10_tf)
        f100_test_mtf.append(temp_f100_tf)
        f1000_test_mtf.append(temp_f1000_tf)


*For Term Frequency we need new dictionary for words*

In [533]:
# occurence
word_occ_spam = {}
word_occ_legit = {}
for word in dist_words:
    word_occ_spam[word] = 0
    word_occ_legit[word] = 0

for path in paths:
    if(path[-2:] == '10'):  
        continue
    files = [os.path.join(path, f) for f in os.listdir(path)]  
    for email in files:
        if(email.find('spmsg') != -1):  
            is_spam = True
        else:
            is_spam = False
        # dist_set = set() # there could be repeated word, we only need  occurance, use set
        with open(email) as m:
            for i, content in enumerate(m):  
                if(i == 2):  
                    words = content.split()
                    for word in words:
                        if(word.isalpha() == False): # skip symbols
                            continue
                        else:
                            if(is_spam == True):
                                word_occ_spam[word] += 1
                            else:
                                word_occ_legit[word] += 1

# total #occurence of word in spam
total_occ_spam = 0
total_occ_legit = 0
for i in word_occ_spam.values():
    total_occ_spam += i
for i in word_occ_legit.values():
    total_occ_legit += i
print(f'total words\' occurence in spam is {total_occ_spam}, total occurence in legit is {total_occ_legit}')    

total words' occurence in spam is 126257, total occurence in legit is 552314


**1. Top 10 feature**

In [534]:
N = 10

1.1 compute probability

*In TF computation, it requires a lot in precision so we need more decimal: import decimal library*

In [535]:
# In TF computation, it requires a lot in precision so we need more decimal
from decimal import *
getcontext().prec = 32
# M = #distinct_term
M = len(dist_words)
# compute P(x|spam)
p_x_spam_10 = []
for case in f10_test_mtf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= Decimal(word_occ_spam[f10[i]] + 1) / Decimal(total_occ_spam + M) # with laplacian smoothing
            # if p == 0 : 
            #     print(word_occ_spam[f10[i]] + 1, " ", total_occ_spam + M)
        else:
            p *= 1 # different from bernoulli: ignore negative evidence 
    p_x_spam_10.append(p)    

# compute P(x|legit)
p_x_legit_10 = []
for case in f10_test_mtf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= Decimal(word_occ_legit[f10[i]] + 1) / Decimal(total_occ_legit + M) # with laplacian smoothing
        else:
            p *= 1 # different from bernoulli: ignore negative evidence
    p_x_legit_10.append(p)    

# compute P(spam)
p_spam = Decimal(spam)/Decimal((spam+legit))

# compute P(legit)
p_legit = Decimal(legit)/Decimal((spam+legit)) 

# compute P(x)
p_x_10 = []
for i in range(len(f10_test_mtf)):
    p = p_spam * p_x_spam_10[i] + p_legit * p_x_legit_10[i]
    # if p == 0:
    #     print(p_spam, " ",p_x_spam_10[i]," ", p_legit," ", p_x_legit_10[i])
    p_x_10.append(p)
 
#print(len(p_x_spam_10))
#print(len(p_x_10))

1.2 Classify

In [536]:
f10_pre = []
p_spam_x_10 = []
p_legit_x_10 = []
for i in range(len(f10_test_mbf)):
    psx = p_x_spam_10[i] * p_spam / p_x_10[i]
    p_spam_x_10.append(psx)
    plx = p_x_legit_10[i] * p_legit / p_x_10[i]
    p_legit_x_10.append(plx)
    if psx >= plx :
        f10_pre.append(1)
    else:
        f10_pre.append(0)

print("samples of P(X|spam) with Decimal: ")
print(p_spam_x_10[:10])

samples of P(X|spam) with Decimal: 
[Decimal('0.99813182451384322042671374197205'), Decimal('0.166026133743274404304381245196'), Decimal('0.0000022034784718669146972481962397807'), Decimal('3.4907685249250161423576733648076E-7'), Decimal('3.4907685249250161423576733648076E-7'), Decimal('0.00028888462741023435198687055770385'), Decimal('5.7340307292691391621873985692292E-8'), Decimal('0.000047464415279492994695890697850526'), Decimal('5.7340307292691391621873985692292E-8'), Decimal('0.00060745961945622674572360319748163')]


1.3 Report result

In [537]:
correct = 0;
total = len(test_label)
for i in range(total):
    if test_label[i] == f10_pre[i]:
        correct+=1
correct_rate = correct / total
print("correct rate: ",correct_rate)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if f10_pre[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_mtf_10 = true_spam_in_spam / total_pre_spam   
print("spam precision: ",spam_precision_mtf_10)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if f10_pre[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_mtf_10 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_mtf_10)

correct rate:  0.9553264604810997
spam precision:  0.8333333333333334
spam_recall:  0.9183673469387755


**2. Top 100 feature**

In [538]:
N = 100

2.1 Compute probability

In [539]:
# In TF computation, it requires a lot in precision so we need more decimal
from decimal import *
getcontext().prec = 32
# M = #distinct_term
M = len(dist_words)
# compute P(x|spam)
p_x_spam_100 = []
for case in f100_test_mtf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= Decimal(word_occ_spam[f100[i]] + 1) / Decimal(total_occ_spam + M) # with laplacian smoothing
            # if p == 0 : 
            #     print(word_occ_spam[f100[i]] + 1, " ", total_occ_spam + M)
        else:
            p *= 1 # different from bernoulli: ignore negative evidence 
    p_x_spam_100.append(p)    

# compute P(x|legit)
p_x_legit_100 = []
for case in f100_test_mtf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= Decimal(word_occ_legit[f100[i]] + 1) / Decimal(total_occ_legit + M) # with laplacian smoothing
        else:
            p *= 1 # different from bernoulli: ignore negative evidence
    p_x_legit_100.append(p)    

# compute P(spam)
p_spam = Decimal(spam)/Decimal((spam+legit))

# compute P(legit)
p_legit = Decimal(legit)/Decimal((spam+legit)) 

# compute P(x)
p_x_100 = []
for i in range(len(f100_test_mtf)):
    p = p_spam * p_x_spam_100[i] + p_legit * p_x_legit_100[i]
    # if p == 0:
    #     print(p_spam, " ",p_x_spam_100[i]," ", p_legit," ", p_x_legit_100[i])
    p_x_100.append(p)
 
#print(len(p_x_spam_100))
#print(len(p_x_100))

2.2 Classify

In [540]:
f100_pre = []
p_spam_x_100 = []
p_legit_x_100 = []
for i in range(len(f100_test_mbf)):
    psx = p_x_spam_100[i] * p_spam / p_x_100[i]
    p_spam_x_100.append(psx)
    plx = p_x_legit_100[i] * p_legit / p_x_100[i]
    p_legit_x_100.append(plx)
    if psx >= plx :
        f100_pre.append(1)
    else:
        f100_pre.append(0)

print("samples of P(X|spam) with Decimal: ")
print(p_spam_x_100[:10])

samples of P(X|spam) with Decimal: 
[Decimal('0.99999999999999998337166666657676'), Decimal('8.7667726792794999330849661998533E-12'), Decimal('4.7808810015814275018609484432725E-15'), Decimal('2.4178519897705639368102547623079E-11'), Decimal('1.2003124232990503545616438244857E-17'), Decimal('5.0123643502595835034544050824268E-20'), Decimal('2.6141063245340595174982451624102E-26'), Decimal('2.8830215514908281028370282449559E-23'), Decimal('1.9001432391592828242709248506245E-16'), Decimal('2.6144283445225336417743383127829E-18')]


2.3 Report result

In [541]:
correct = 0;
total = len(test_label)
for i in range(total):
    if test_label[i] == f100_pre[i]:
        correct+=1
correct_rate = correct / total
print("correct rate: ",correct_rate)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if f100_pre[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_mtf_100 = true_spam_in_spam / total_pre_spam   
print("spam precision: ",spam_precision_mtf_100)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if f100_pre[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_mtf_100 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_mtf_100)

correct rate:  0.9862542955326461
spam precision:  0.9591836734693877
spam_recall:  0.9591836734693877


**3. Top 1000 feature**

In [542]:
N = 1000

3.1 Compute probability

In [543]:
# In TF computation, it requires a lot in precision so we need more decimal
from decimal import *
getcontext().prec = 32
# M = #distinct_term
M = len(dist_words)
# compute P(x|spam)
p_x_spam_1000 = []
for case in f1000_test_mtf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= Decimal(word_occ_spam[f1000[i]] + 1) / Decimal(total_occ_spam + M) # with laplacian smoothing
            # if p == 0 : 
            #     print(word_occ_spam[f1000[i]] + 1, " ", total_occ_spam + M)
        else:
            p *= 1 # different from bernoulli: ignore negative evidence 
    p_x_spam_1000.append(p)    

# compute P(x|legit)
p_x_legit_1000 = []
for case in f1000_test_mtf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= Decimal(word_occ_legit[f1000[i]] + 1) / Decimal(total_occ_legit + M) # with laplacian smoothing
        else:
            p *= 1 # different from bernoulli: ignore negative evidence
    p_x_legit_1000.append(p)    

# compute P(spam)
p_spam = Decimal(spam)/Decimal((spam+legit))

# compute P(legit)
p_legit = Decimal(legit)/Decimal((spam+legit)) 

# compute P(x)
p_x_1000 = []
for i in range(len(f1000_test_mtf)):
    p = p_spam * p_x_spam_1000[i] + p_legit * p_x_legit_1000[i]
    # if p == 0:
    #     print(p_spam, " ",p_x_spam_1000[i]," ", p_legit," ", p_x_legit_1000[i])
    p_x_1000.append(p)
 
#print(len(p_x_spam_1000))
#print(len(p_x_1000))

3.2 Classify

In [544]:
f1000_pre = []
p_spam_x_1000 = []
p_legit_x_1000 = []
for i in range(len(f1000_test_mbf)):
    psx = p_x_spam_1000[i] * p_spam / p_x_1000[i]
    p_spam_x_1000.append(psx)
    plx = p_x_legit_1000[i] * p_legit / p_x_1000[i]
    p_legit_x_1000.append(plx)
    if psx >= plx :
        f1000_pre.append(1)
    else:
        f1000_pre.append(0)

print("samples of P(X|spam) with Decimal: ")
print(p_spam_x_1000[:10])

samples of P(X|spam) with Decimal: 
[Decimal('1'), Decimal('2.1379484351764377091582505756035E-33'), Decimal('2.3704402227890955375887136402456E-39'), Decimal('4.7433400474497834382595792823822E-46'), Decimal('2.6854239423586455534220495287374E-67'), Decimal('1.6340586087808201789776181084828E-94'), Decimal('3.1013995372555734938318883288304E-66'), Decimal('1.2347449703631891006733487498684E-85'), Decimal('6.1823061090089771542762569055904E-37'), Decimal('2.2495019925776210841146498096943E-51')]


3.3 Report

In [545]:
correct = 0;
total = len(test_label)
for i in range(total):
    if test_label[i] == f1000_pre[i]:
        correct+=1
correct_rate = correct / total
print("correct rate: ",correct_rate)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if f1000_pre[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_mtf_1000 = true_spam_in_spam / total_pre_spam   
print("spam precision: ",spam_precision_mtf_1000)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if f1000_pre[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_mtf_1000 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_mtf_1000)

correct rate:  0.9896907216494846
spam precision:  1.0
spam_recall:  0.9387755102040817


# Part2 report: spam precision and spam recall of combinations of NB filters and top-N feature

In [546]:
print(f"Bernoulli NB with binary feature and top-10: precesion = {spam_precision_bbf_10}, recall = {spam_recall_bbf_10}")
print(f"Bernoulli NB with binary feature and top-100: precesion = {spam_precision_bbf_100}, recall = {spam_recall_bbf_100}")
print(f"Bernoulli NB with binary feature and top-1000: precesion = {spam_precision_bbf_1000}, recall = {spam_recall_bbf_1000}")
print(f"Multinomial NB with binary feature and top-10: precesion = {spam_precision_mbf_10}, recall = {spam_recall_mbf_10}")
print(f"Multinomial NB with binary feature and top-100: precesion = {spam_precision_mbf_100}, recall = {spam_recall_mbf_100}")
print(f"Multinomial NB with binary feature and top-1000: precesion = {spam_precision_mbf_1000}, recall = {spam_recall_mbf_1000}")
print(f"Multinomial NB with term frequency and top-10: precesion = {spam_precision_mtf_10}, recall = {spam_recall_mtf_10}")
print(f"Multinomial NB with term frequency and top-100: precesion = {spam_precision_mtf_100}, recall = {spam_recall_mtf_100}")
print(f"Multinomial NB with term frequency and top-1000: precesion = {spam_precision_mtf_1000}, recall = {spam_recall_mtf_1000}")

Bernoulli NB with binary feature and top-10: precesion = 0.8888888888888888, recall = 0.8163265306122449
Bernoulli NB with binary feature and top-100: precesion = 1.0, recall = 0.673469387755102
Bernoulli NB with binary feature and top-1000: precesion = 1.0, recall = 0.6122448979591837
Multinomial NB with binary feature and top-10: precesion = 0.8695652173913043, recall = 0.8163265306122449
Multinomial NB with binary feature and top-100: precesion = 0.9038461538461539, recall = 0.9591836734693877
Multinomial NB with binary feature and top-1000: precesion = 1.0, recall = 0.9387755102040817
Multinomial NB with term frequency and top-10: precesion = 0.8333333333333334, recall = 0.9183673469387755
Multinomial NB with term frequency and top-100: precesion = 0.9591836734693877, recall = 0.9591836734693877
Multinomial NB with term frequency and top-1000: precesion = 1.0, recall = 0.9387755102040817


# SVM based filter

load data, generate top-N feature matrix with IG

In [547]:
train_path = './lingspam_public/lemm_stop'  # parent directory of dataset we use
paths = [os.path.join(train_path, folders) for folders in os.listdir(train_path)]  
all_words = [] 

train_data = [] # all feature
train_data_10 = []
train_data_100 = []
train_data_1000 = []
train_label = []

test_data = [] # all feature
test_data_10 = []
test_data_100 = []
test_data_1000 = []
test_label = []



for path in paths:
    files = [os.path.join(path, f) for f in os.listdir(path)]  # get file names of this part and generate path to files
    for email in files:
        if email.find('spmsg') != -1 : #check filenames to check if it is spam 
            if path[-2:] == '10' : # if it is test data
                test_label.append(1)
            else:
                train_label.append(1)
        else:
            if path[-2:] == '10' :
                test_label.append(0)
            else:
                train_label.append(0)
        
        with open(email) as m:
            for i, content in enumerate(m):  # i: line number , content: content of each line
                if(i == 2):  # open txt and find content is in the 3rd line
                    words_in_email = set(content.split())
                    temp_train_10 = []
                    temp_train_100 = []
                    temp_train_1000 = []
                    temp_test_10 = []
                    temp_test_100 = []
                    temp_test_1000 = []

                    temp_train = []
                    temp_test = []
                    for w in dist_words:
                        if w in words_in_email:
                            if path[-2:] == '10' :
                                temp_test.append(1)
                            else:
                                temp_train.append(1)
                        else:
                            if path[-2:] == '10' :
                                temp_test.append(0)
                            else:
                                temp_train.append(0)
                    for w in f10:
                        if w in words_in_email:
                            if path[-2:] == '10' :
                                temp_test_10.append(1)
                            else:
                                temp_train_10.append(1)
                        else:
                            if path[-2:] == '10' :
                                temp_test_10.append(0)
                            else:
                                temp_train_10.append(0)
                    for w in f100:
                        if w in words_in_email:
                            if path[-2:] == '10' :
                                temp_test_100.append(1)
                            else:
                                temp_train_100.append(1)
                        else:
                            if path[-2:] == '10' :
                                temp_test_100.append(0)
                            else:
                                temp_train_100.append(0)
                    for w in f1000:
                        if w in words_in_email:
                            if path[-2:] == '10' :
                                temp_test_1000.append(1)
                            else:
                                temp_train_1000.append(1)
                        else:
                            if path[-2:] == '10' :
                                temp_test_1000.append(0)
                            else:
                                temp_train_1000.append(0)

        if path[-2:] == '10':
            test_data.append(np.array(temp_test))
            test_data_10.append(np.array(temp_test_10))
            test_data_100.append(np.array(temp_test_100))
            test_data_1000.append(np.array(temp_test_1000))
        else :
            train_data.append(np.array(temp_train))
            train_data_10.append(np.array(temp_train_10))
            train_data_100.append(np.array(temp_train_100))
            train_data_1000.append(np.array(temp_train_1000))


*Memo for an exception : when I try to fit for SVM, it says "setting an array element with a sequence" which indicates my matrix is not a generalized "rectangular", then I found I mistakenly wrote temp_train_10.append() in a top-1000 situation(part10).*

**ALL feature**

*Memo: Be carefull with strength of regularization,  make it soft because what I have is 0-1 feature!*

In [548]:
from sklearn import svm
svm_ = svm.SVC(kernel = 'rbf',C=1000,probability=False) # because it is 0-1 feature, no need for strength on regularization
%time svm_.fit(train_data,train_label)

CPU times: user 2min 30s, sys: 91.1 ms, total: 2min 30s
Wall time: 2min 30s


SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [549]:
pre_svm = svm_.predict(test_data)
print(pre_svm)

[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0]


In [550]:
total = len(test_label)
# correct rate
score = svm_.score(test_data, test_label)
print("correct rate: ",score)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if pre_svm[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_svm_af = true_spam_in_spam / (total_pre_spam) 
print("spam precision: ",spam_precision_svm_af)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if pre_svm[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_svm_af = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_svm_af)

correct rate:  0.9828178694158075
spam precision:  1.0
spam_recall:  0.8979591836734694


**1. Top 10 feature**

In [551]:
from sklearn import svm
#svm_10 = svm.SVC(kernel = 'linear',C=1000,probability=False)
svm_10 = svm.SVC(kernel = 'rbf',C=1000,probability=False)
%time svm_10.fit(train_data_10,train_label)

CPU times: user 40.1 ms, sys: 977 µs, total: 41 ms
Wall time: 42.5 ms


SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [552]:
pre_svm_10 = svm_10.predict(test_data_10)
print(pre_svm_10)

[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0]


In [553]:
total = len(test_label)
# correct rate
score_10 = svm_10.score(test_data_10, test_label)
print("correct rate: ",score_10)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if pre_svm_10[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_svm_10 = true_spam_in_spam / (total_pre_spam) 
print("spam precision: ",spam_precision_svm_10)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if pre_svm_10[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_svm_10 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_svm_10)

correct rate:  0.9450171821305842
spam precision:  0.851063829787234
spam_recall:  0.8163265306122449


**2. Top 100 feature**

In [554]:
from sklearn import svm
#svm_100 = svm.SVC(kernel = 'linear',C=1000,probability=False)
svm_100 = svm.SVC(kernel = 'rbf',C=1000,probability=False)
%time svm_100.fit(train_data_100,train_label)

CPU times: user 173 ms, sys: 2.99 ms, total: 176 ms
Wall time: 175 ms


SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [555]:
pre_svm_100 = svm_100.predict(test_data_100)
print(pre_svm_100)

[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0]


In [556]:
total = len(test_label)
# correct rate
score_100 = svm_100.score(test_data_100, test_label)
print("correct rate: ",score_100)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if pre_svm_100[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_svm_100 = true_spam_in_spam / (total_pre_spam) 
print("spam precision: ",spam_precision_svm_100)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if pre_svm_100[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_svm_100 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_svm_100)

correct rate:  0.9690721649484536
spam precision:  0.9545454545454546
spam_recall:  0.8571428571428571


**3. Top 1000 feature**

In [557]:
from sklearn import svm
#svm_1000 = svm.SVC(kernel = 'linear',C=1000,probability=False)
svm_1000 = svm.SVC(kernel = 'rbf',C=1000,probability=False)
%time svm_1000.fit(train_data_1000,train_label)

CPU times: user 2.39 s, sys: 4.01 ms, total: 2.39 s
Wall time: 2.4 s


SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [558]:
pre_svm_1000 = svm_1000.predict(test_data_1000)
print(pre_svm_1000)

[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1
 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0]


In [559]:
total = len(test_label)
# correct rate
score_1000 = svm_1000.score(test_data_1000, test_label)
print("correct rate: ",score_1000)

# spam precision is defined as the fraction of true spam e-mails among all e-mails predicted as spam
total_pre_spam = 0
true_spam_in_spam = 0
for i in range(total):
    if pre_svm_1000[i] == 1:
        total_pre_spam += 1
        if test_label[i] == 1:
            true_spam_in_spam += 1
spam_precision_svm_1000 = true_spam_in_spam / (total_pre_spam) 
print("spam precision: ",spam_precision_svm_1000)

# spam recall is defined as fraction of true spam e-mails predicted as spam.
pre_spam_in_true = 0
total_true_spam = 0
for i in range(total):
    if test_label[i] == 1:
        total_true_spam += 1
        if pre_svm_1000[i] == 1:
            pre_spam_in_true += 1
        
spam_recall_svm_1000 = pre_spam_in_true / total_true_spam  
print("spam_recall: ",spam_recall_svm_1000)

correct rate:  0.9862542955326461
spam precision:  0.9787234042553191
spam_recall:  0.9387755102040817


# Part4 Report: SVM spam precision and spam recall

I used SVM with RBF kernel, it perform obviously better on recall than linear kernel. And I set regularization very soft because I am using binary feature, it only has 0 or 1. 

For features, I used binary word feature, and select them by IG used before, I tried all feature and top-10, top-100, top-1000 from IG.

**It results into top-1000 with RBF kernel performs best in general, because its precision and recall rate are both high.**

In [560]:
print(f"RBF kernal with all feature: precision = {spam_precision_svm_af}, recall = {spam_recall_svm_af}")
print(f"RBF kernal with top-10 feature: precision = {spam_precision_svm_10}, recall = {spam_recall_svm_10}")
print(f"RBF kernal with top-100 feature: precision = {spam_precision_svm_100}, recall = {spam_recall_svm_100}")
print(f"RBF kernal with top-1000 feature: precision = {spam_precision_svm_1000}, recall = {spam_recall_svm_1000}")

RBF kernal with all feature: precision = 1.0, recall = 0.8979591836734694
RBF kernal with top-10 feature: precision = 0.851063829787234, recall = 0.8163265306122449
RBF kernal with top-100 feature: precision = 0.9545454545454546, recall = 0.8571428571428571
RBF kernal with top-1000 feature: precision = 0.9787234042553191, recall = 0.9387755102040817


Some extra statistics when using linear kernel:

linear & top-10: precision = 0.88372, recall = 0.775510

linear & top-100: precision = 0.9743, recall = 0.775510

linear & top-1000: precision = 1.0, recall = 0.85714

# Adversarial Classification

baseline multinomial NB with binary feature and top-10 feature

**1. Before attacker's modification**

In [561]:
N = 10
test_label = []  
f10_test_mbf = []  # test emails' feature matrix for top 10

emails = [os.path.join(test_path, file_name) for file_name in os.listdir(test_path)]  
for email in emails:
    # spam: 1, legit: 0
    if email.find('spmsg')!=-1:
        test_label.append(1)
    else:
        test_label.append(0)
    
    with open(email) as m:
        temp_f10_bf = []
        for i, content in enumerate(m):  
            if i==2:  # content at 3rd line
                words_in_email = set(content.split())
                for w in f10:
                    if w in words_in_email:
                        temp_f10_bf.append(1)
                    else:
                        temp_f10_bf.append(0)

        f10_test_mbf.append(temp_f10_bf)
        
# compute P(x|spam)
p_x_spam_10 = []
for case in f10_test_mbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_spam[f10[i]] + 1) / (spam + 2) 
        else:
            p *= 1 
    p_x_spam_10.append(p)    

# compute P(x|legit)
p_x_legit_10 = []
for case in f10_test_mbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_legit[f10[i]] + 1) / (legit + 2) 
        else:
            p *= 1 
    p_x_legit_10.append(p)    

# compute P(spam)
p_spam = spam/(spam+legit)

# compute P(legit)
p_legit = legit/(spam+legit) 

# compute P(x)
p_x_10 = []
for i in range(len(f10_test_mbf)):
    p = p_spam * p_x_spam_10[i] + p_legit * p_x_legit_10[i]
    p_x_10.append(p)
 
#print(len(p_x_spam_10))
#print(len(p_x_10))

f10_pre = []
p_spam_x_10 = []
p_legit_x_10 = []
for i in range(len(f10_test_mbf)):
    psx = p_x_spam_10[i] * p_spam / p_x_10[i]
    p_spam_x_10.append(psx)
    plx = p_x_legit_10[i] * p_legit / p_x_10[i]
    p_legit_x_10.append(plx)
    if psx >= plx :
        f10_pre.append(1)
    else:
        f10_pre.append(0)

In [562]:
# false negative
fn_before = 0
true_positive = 0
for i in range(total):
    if test_label[i] == 1 and f10_pre[i] == 1 :
        true_positive +=1
    if test_label[i] == 1 and f10_pre[i] == 0:
        fn_before+= 1
        
fnr_before = n_fn_before / (true_positive + n_fn_before)  
print("Flase negative rate before attacker's modification: ",fnr_before) 

Flase negative rate before attacker's modification:  0.1836734693877551


**2. After attacker's modification**

Compute P(Xi=xi|C=c)

In [563]:
# compute P(x|spam)

p_1_spam_10 = []
p_0_spam_10 = []

for i in range(N):
    # 1_spam
    p = (word_spam[f10[i]] + 1) / (spam + 2) # with laplacian smoothing
    p_1_spam_10.append(p)
    # 0_spam
    p = 1 - (word_spam[f10[i]] + 1) / (spam + 2)   
    p_0_spam_10.append(p) 

# compute P(x|legit)
p_1_legit_10 = []
p_0_legit_10 = []
p = 1
for i in range(N):
    # 1_spam
    p = (word_legit[f10[i]] + 1) / (legit + 2) # with laplacian smoothing
    p_1_legit_10.append(p)
    # 0_spam
    p = 1 - (word_legit[f10[i]] + 1) / (legit + 2)
    p_0_legit_10.append(p)  

Compute desired reduction log( P(spam | x) / P(legit | x) ) =  log( P(x | spam) / P(x | legit) ) + log(P(spam)/P(legit))

In [564]:
desired_delta = []
for case in f10_test_mbf:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= p_1_spam_10[i] / p_1_legit_10[i]
        else:
            p *= p_0_spam_10[i] / p_0_legit_10[i]
    d = np.log2(p) + np.log2(spam/legit)
    desired_delta.append(d)

print("desired delta:",desired_delta[5:15])
print("corresponding label:",test_label[5:15])

desired delta: [-12.546866882101583, -24.16676583757637, -15.405885725154178, -24.16676583757637, -10.254593464194329, 5.0048781098057695, -14.629465573740717, -24.16676583757637, -19.123452183326485, -15.405885725154178]
corresponding label: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


compute  delta_LO_ix' = LOc(xi) - LOc(xi') for top-10 features

LO(xi) = log( P(xi | spam) / P(xi | legit) )

As assignment states, the adversary only use **ADD_WORDS** strategy, so I set reduction for term_i to 0 when each email has this term

Only consider adding word for reduction of each feature 

In [565]:
delta_add_word = []
for i in range(10):
    d = np.log2(p_0_spam_10[i] / p_0_legit_10[i]) - np.log2(p_1_spam_10[i] / p_1_legit_10[i]) # delta_LO_ix' = LOc(xi) - LOc(xi') old - new
    d = max(0,d)
    delta_add_word.append(d)

determine reduction acquired and **sort them** by value, now we can see top-10words in this sorted order

In [566]:
sorted_delta = []
for case in f10_test_mbf:
    delta_i = {}
    for i in range(10):
        if case[i] == 1:
            delta_i[f10[i]] = 0
        else:
            delta_i[f10[i]] = delta_add_word[i]
    sorted_delta_i = OrderedDict(sorted(delta_i.items(), key=lambda x : x[1], reverse=True))
    sorted_delta.append(sorted_delta_i)

#print("check f_index helper: index of remove",f_index["remove"])
print("orgin f10 sequence: ", f10)
#print("case 0:", f10_test_mbf[0])
print("desired delta of case 0: ",desired_delta[0])
for (w,d) in sorted_delta[0].items():
    print(w,":", d)

orgin f10 sequence:  ['language', 'remove', 'free', 'linguistic', 'university', 'money', 'click', 'market', 'our', 'business']
desired delta of case 0:  7.430667122628313
linguistic : 8.760880112422193
language : 6.678281420783058
university : 5.043313654249885
remove : 0
free : 0
money : 0
click : 0
market : 0
our : 0
business : 0


Because adding a term incur an unit cost, we can pick term with larger reduction first (*being greedy*) for minimum cost tring to meet desired reduction! 

**Start modify**

In [567]:
# feature index helper
f_index = {}
for i in range(10):
    f_index[f10[i]]=i

avg_cost = 0
test_spam = 0
f10_test_mbf_mod = []
for i in f10_test_mbf:
    f10_test_mbf_mod.append(i.copy())

for i in range(len(sorted_delta)):
    if test_label[i] == 0: # do not modify legit email
        continue
    else:
        test_spam+=1
    d = sorted_delta[i]
    sum = 0
    for (k,v) in d.items():
        if v <= 0: #if value reach 0 in sorted reduction values, can't reach desired reduction, break
            break
        else: # reduction > 0
            f10_test_mbf_mod[i][f_index[k]] = 1 # add word
            avg_cost+=1
            sum += v
            if desired_delta[i] <= sum: # meet desired reduction 
                break; 
avg_cost /= test_spam
print("average cost of attacker: ", avg_cost)                

average cost of attacker:  1.469387755102041


In [568]:
print(f10)
print("origin:",f10_test_mbf[0])
print("modify:",f10_test_mbf_mod[0])

['language', 'remove', 'free', 'linguistic', 'university', 'money', 'click', 'market', 'our', 'business']
origin: [0, 0, 1, 0, 0, 0, 0, 0, 1, 1]
modify: [0, 0, 1, 1, 0, 0, 0, 0, 1, 1]


perform classification after attacker's modification on test data

In [569]:
# compute P(x|spam)
p_x_spam_10 = []
for case in f10_test_mbf_mod:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_spam[f10[i]] + 1) / (spam + 2) # with laplacian smoothing
        else:
            p *= 1 
    p_x_spam_10.append(p)    

# compute P(x|legit)
p_x_legit_10 = []
for case in f10_test_mbf_mod:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_legit[f10[i]] + 1) / (legit + 2) # with laplacian smoothing
        else:
            p *= 1 
    p_x_legit_10.append(p)    

# compute P(x)
p_x_10 = []
for i in range(len(f10_test_mbf_mod)):
    p = p_spam * p_x_spam_10[i] + p_legit * p_x_legit_10[i]
    p_x_10.append(p)

f10_mod_pre = []
p_spam_x_10 = []
p_legit_x_10 = []
for i in range(len(f10_test_mbf_mod)):
    psx = p_x_spam_10[i] * p_spam / p_x_10[i]
    p_spam_x_10.append(psx)
    plx = p_x_legit_10[i] * p_legit / p_x_10[i]
    p_legit_x_10.append(plx)
    if psx >= plx :
        f10_mod_pre.append(1)
    else:
        f10_mod_pre.append(0)

In [570]:
# false negative
fn_after = 0
true_positive = 0
for i in range(total):
    if test_label[i] == 1 and f10_mod_pre[i] == 1 :
        true_positive +=1
    if test_label[i] == 1 and f10_mod_pre[i] == 0:
        fn_after+= 1
        
fnr_after = fn_after / (true_positive + fn_after)  
print("Flase negative rate after attacker's modification: ",fnr_after) 

Flase negative rate after attacker's modification:  0.9183673469387755


**3. After Update calssifier**

PA{spam|x'}/PA{legit|x'} = PA{x'|spam}*PA{SPAM} / (PA{x'|legit} * PA{legit})

In [571]:
# # compute PA(x'|spam), x' can be generated by any possible x (ADD WORDS)so it is sum of P(x|spam)
# p_x_spam_10 = []
# for CASE in f10_test_mbf_mod: # facing attack
#     sum = 0
#     for case in f10_test_mbf_mod:
#         # can contain it self I think, or there might be zero availble x
#         if case == CASE:
#             continue
#         candidate = True
#         count_add = 0
#         for i in range(len(case)):
#             # because we assume classifier knows adversary will add word in a optimal way, so according to average cost of attack, I set limit to at most 1 word to add
#             if count_add > 1:  
#                 candidate = False
#             if CASE[i] == 0 and case[i] == 1:
#                 candidate = False
#             else: # can ADD WORDS to case to get CASE
#                 if CASE[i] == 1 and case[i] == 0:
#                     count_add+=1
#         p = 0
#         if candidate == True:
#             p = 1
#             for i in range(N):
#                 if case[i] == 1:
#                     p *= (word_spam[f10[i]] + 1) / (spam + 2) 
#                 else:
#                     p *= 1   
#         sum += p
#     p_x_spam_10.append(sum)    

# # compute PA(x'|legit)
# p_x_legit_10 = []
# for case in f10_test_mbf_mod:
#     p = 1
#     for i in range(N):
#         if case[i] == 1:
#             p *= (word_legit[f10[i]] + 1) / (legit + 2) 
#         else:
#             p *= 1
#     p_x_legit_10.append(p)    



# f10_update_pre = []

# for i in range(len(f10_test_mbf_mod)):
#     psx = p_x_spam_10[i] * p_spam 
#     p_spam_x_10.append(psx)
#     plx = p_x_legit_10[i] * p_legit 
#     p_legit_x_10.append(plx)
#     if psx / plx >=1 :
#         f10_update_pre.append(1)
#     else:
#         f10_update_pre.append(0)

In [572]:
# compute PA(x'|legit)
p_x_legit_10 = []
for case in f10_test_mbf_mod:
    p = 1
    for i in range(N):
        if case[i] == 1:
            p *= (word_legit[f10[i]] + 1) / (legit + 2) 
        else:
            p *= 1
    p_x_legit_10.append(p) 

# assume classifier know attack strategy: always add words with minimum cost
# compute PA(x'|spam), x' can be generated by any possible x (ADD WORDS), so it is sum of P(x|spam)
p_x_spam_10 = []
for i in range(len(f10_test_mbf_mod)):
    case = f10_test_mbf_mod[i] 
    sum = 0
    for w in sorted_delta[i].keys(): # according to sorted delta(reduction), inversely find optimal original x from x' by remove word with larger delta (greedy)
        if case[f_index[w]] == 1: # can remove word
            case[f_index[w]] = 0  # remove
            psx = 1
            plx = 1
            for i in range(N):
                if case[i] == 1:
                    psx *= (word_spam[f10[i]] + 1) / (spam + 2) 
                else:
                    psx *= 1
                if case[i] == 1:
                    plx *= (word_legit[f10[i]] + 1) / (legit + 2) 
                else:
                    plx *= 1
            if psx*p_spam / (plx*p_legit) >= 1: # the original one inversly recover from modified case is spam! we find attacker's original case to modify
                sum+=psx
                break
            else:
                 continue
    p_x_spam_10.append(sum)

f10_update_pre = []

for i in range(len(f10_test_mbf_mod)):
    psx = p_x_spam_10[i] * p_spam 
    p_spam_x_10.append(psx)
    plx = p_x_legit_10[i] * p_legit 
    p_legit_x_10.append(plx)
    if psx / plx >=1 :
        f10_update_pre.append(1)
    else:
        f10_update_pre.append(0)

In [573]:
# false negative
fn_update = 0
true_positive = 0
for i in range(total):
    if test_label[i] == 1 and f10_update_pre[i] == 1 :
        true_positive +=1
    if test_label[i] == 1 and f10_update_pre[i] == 0:
        fn_update+= 1
        
fnr_update = fn_update / (true_positive + fn_update)  
print("Flase negative rate after update classifier: ",fnr_update) 

# false positive
fp_update = 0
true_negative = 0
for i in range(total):
    if test_label[i] == 0 and f10_update_pre[i] == 0 :
        true_negative +=1
    if test_label[i] == 0 and f10_update_pre[i] == 1:
        fp_update+= 1
        
fpr_update = fp_update / (true_negative + fp_update)  
print("Flase positive rate after update classifier: ",fpr_update) 

Flase negative rate after update classifier:  0.1836734693877551
Flase positive rate after update classifier:  0.17768595041322313


# Part4 report: adversarial classification

In [574]:
print("Flase negative rate before attacker's modification: ",fnr_before) 
print("Flase negative rate after attacker's modification: ",fnr_after)
print("average cost of attacker: ", avg_cost)  
print("Flase negative rate after update classifier: ",fnr_update)   
print("Flase positive rate after update classifier: ",fpr_update) 

Flase negative rate before attacker's modification:  0.1836734693877551
Flase negative rate after attacker's modification:  0.9183673469387755
average cost of attacker:  1.469387755102041
Flase negative rate after update classifier:  0.1836734693877551
Flase positive rate after update classifier:  0.17768595041322313
